In [1]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [2]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
340,Directed by Diane Keaton and adapted from a bo...,negative
709,I don't understand the low 5.7 rating on this ...,positive
87,"""Don't be greedy"" sums up the depth of this mo...",negative
318,I saw this film on True Movies (which automati...,positive
866,I had a heck of a good time viewing this pictu...,positive


In [3]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [4]:
df = normalize_text(df)
df.head()

,review,sentiment
340,directed diane keaton adapted book franz lidz ...,negative
709,understand low rating film delight people like...,positive
87,don t greedy sum depth movie rest baloney big ...,negative
318,saw film true movie which automatically made s...,positive
866,heck good time viewing picture splendidly surp...,positive


In [5]:
df['sentiment'].value_counts()

sentiment
negative    273
positive    227
Name: count, dtype: int64

In [6]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [7]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
340,directed diane keaton adapted book franz lidz ...,0
709,understand low rating film delight people like...,1
87,don t greedy sum depth movie rest baloney big ...,0
318,saw film true movie which automatically made s...,1
866,heck good time viewing picture splendidly surp...,1


In [8]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [9]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [16]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/nagarajloni209/Capstone-Project-MLOPs-.mlflow')
dagshub.init(repo_owner='nagarajloni209', repo_name='Capstone-Project-MLOPs', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


Initialized MLflow to track repo "nagarajloni209/Capstone-Project-MLOPs"

Repository nagarajloni209/Capstone-Project-MLOPs initialized!

<Experiment: artifact_location='mlflow-artifacts:/9a5c23b4aa034044bc60ca0a15d9e76e', creation_time=1756745573178, experiment_id='0', last_update_time=1756745573178, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [17]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-09-01 22:23:35,968 - INFO - Starting MLflow run...
2025-09-01 22:23:37,121 - INFO - Logging preprocessing parameters...
2025-09-01 22:23:38,859 - INFO - Initializing Logistic Regression model...
2025-09-01 22:23:38,861 - INFO - Fitting the model...
2025-09-01 22:23:38,904 - INFO - Model training complete.
2025-09-01 22:23:38,906 - INFO - Logging model parameters...
2025-09-01 22:23:39,431 - INFO - Making predictions...
2025-09-01 22:23:39,434 - INFO - Calculating evaluation metrics...
2025-09-01 22:23:39,460 - INFO - Logging evaluation metrics...
2025-09-01 22:23:45,925 - INFO - Saving and logging the model...
2025/09/01 22:23:45 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025-09-01 22:23:47,156 - ERROR - An error occurred: INTERNAL_ERROR: Response: {'error': 'unsupported endpoint, please contact support@dagshub.com'}
Traceback (most recent call last):
  File "C:\Users\nagar\AppData\Local\Temp\ipykernel_28112\2979654972.py", line 48, in 

🏃 View run abundant-ape-679 at: https://dagshub.com/nagarajloni209/Capstone-Project-MLOPs-.mlflow/#/experiments/0/runs/2626bb693bb74effab6d20a11426d219
🧪 View experiment at: https://dagshub.com/nagarajloni209/Capstone-Project-MLOPs-.mlflow/#/experiments/0
